In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('241205_03_reduce')
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 10:48:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Reduce : 여러개의 값을 하나로 축약
RDD.reduce(<func>)

In [58]:
sample_rdd = spark.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[97] at readRDDFromFile at PythonRDD.scala:274

In [59]:
from operator import add
sample_rdd.reduce(add)

15

In [60]:
sample_rdd.reduce(lambda x, y: (x*2) + y)

57

# partition이 나뉘어져 있을 때

lambda x, y : (x*2) + y

1. x=1, y=2 >> (1*2) + 2 = 4
2. x=4, y=3 >> (4*2) + 3 = 11
3. x=11, y =4 >> (11*2) + 4 = 26

In [4]:
sample_rdd1 = spark.parallelize([1,2,3,4])
sample_rdd1.reduce(lambda x, y: (x*2) + y)

26

In [5]:
sample_rdd1.glom().collect()

[[1, 2, 3, 4]]

In [6]:
sample_rdd2 = spark.parallelize([1,2,3,4,], 2) 
sample_rdd2.glom().collect()

[[1, 2], [3, 4]]

In [7]:
sample_rdd2.reduce(lambda x, y: (x*2) + y)

18

In [8]:
sample_rdd3 = spark.parallelize([1, 2, 3, 4], 3)
sample_rdd3.glom().collect()

[[1], [2], [3, 4]]

In [9]:
sample_rdd3.reduce(lambda x, y: (x*2) + y)

18

### 리듀스 연산은 순서 의존적
각 파티션 내에서 연산은 독립적으로 실행, 최종 결과는 파티션 간의 결과가 결합

## fold 연산의 활용
fold(zeroValue, <func>)
    
zeroValue : 시작값, 항등원 *일때 1, +일 경우 0

In [12]:
rdd4= spark.parallelize([2,3,4],4)
rdd4.reduce(lambda x,y : x*y)

24

In [13]:
rdd4.fold(1, lambda x, y : x * y)

24

In [15]:
# 빈 RDD
rdd5 = spark.parallelize([])
try:
    rdd5.reduce(lambda x, y : x + y)
except ValueError as e:
    print(f'Reduce Error : {e}')

Reduce Error : Can not reduce() empty RDD


In [18]:
rdd5.fold(0, lambda x, y : x + y)

0

## reduce(), fold() 비슷한 처리를 합니다.
1. reduce()는 단순한 RDD 축약연산, 비어있지 않은 RDD의 경우
2. fold()는 일반적이고 오류가 없는 연산, 초기값을 적용해서 비어있는 RDD, 파티션의 경우에도 적용할 수 있음

In [19]:
rdd6 = spark.parallelize([1,'a', 2])
rdd6.collect()

[1, 'a', 2]

In [24]:
from py4j.protocol import Py4JJavaError
try:
    res = rdd6.reduce(lambda x, y : str(x)+ str(y))
    print(res)
except Py4JJavaError as e:
    print(f'Reduce Error : {e}')

1a2


In [23]:
rdd6.fold('', lambda x, y :str(x)+ str(y))

'1a2'

In [ ]:
# a*b 파티션 처리

In [26]:
# groupBy(), aggregate()
rdd7 = spark.parallelize([1,2,1,1,3,5,5,8])
rdd7.collect()

[1, 2, 1, 1, 3, 5, 5, 8]

In [28]:
result1 = rdd7.groupBy(lambda x : x % 2).collect()

In [32]:
sorted([(x, sorted(y)) for (x, y) in result1])

[(0, [2, 8]), (1, [1, 1, 1, 3, 5, 5])]

rdd7.aggregate(zeroValue, func1_partion, func2_combine)

In [34]:
func1 = lambda x, y : (x[0]+y, x[1]+1)

In [35]:
func2 = lambda x, y : (x[0]+y[0], x[1]+y[1])

In [37]:
rdd8 = spark.parallelize([1,2,3,4], 2)
rdd8.collect()

[1, 2, 3, 4]

In [38]:
rdd8.aggregate((0, 0), func1, func2)

(10, 4)

In [39]:
rdd8.glom().collect()

[[1, 2], [3, 4]]

p1<br>
x = [1,2], zerovalue = [0,0]<br>
x[0] = 0, x[1] =0<br>
x[0] + y = 0 + 1 = 1, x[1] + 1 = 0 + 1 = 1 >> (1,1)<br>
x[0] + y = 1 + 2 = 3, x[1] + 1 = 1 + 1 = 2 >> (3, 2)<br>

p2<br>
x = [3,4], zerovalue = [0,0]<br>
x[0] = 0, x[1] = 0<br>
x[0] + y = 0 + 3 = 3, x[1] + 1 = 0 + 1 = 4 >> (3,4)<br>
x[0] + y = 3 + 4 = 3, x[1] + 1 = 1 + 1 = 2 >> (7,2)<br>

p1, p2<br>
x = (3, 2), y = (7,2)<br>
x[0] + y[0] = 3+7 = 10, x[1] + y[1] = 2+2 = 4<br>
(10,4)

## key-value RDD (Paired RDD)

groupByKey(), reduceByKey()
그룹핑한 후에 특정 트랜스포메이션을 수행, 키가 필요함.

In [40]:
rdd9 = spark.parallelize(
    [
        ('짜장면', 15),
        ('짬뽕', 10),
        ('짜장면', 5),
        ('짬뽕', 20)
    ]
)

In [41]:
rdd9_g = rdd9.groupByKey()
rdd9_g

PythonRDD[49] at RDD at PythonRDD.scala:53

In [42]:
rdd9_g.mapValues(len).collect() # max, min

[('짜장면', 2), ('짬뽕', 2)]

In [43]:
rdd9_g.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10, 20])]

In [44]:
rdd9_g.mapValues(lambda x: sum(x) / len(x)).collect()

[('짜장면', 10.0), ('짬뽕', 15.0)]

In [45]:
rdd9.reduceByKey(add).collect()

[('짜장면', 20), ('짬뽕', 30)]

In [46]:
# reduceByKey()가 groupByKey()보다 더 빠르다

In [ ]:
# countByKey()
각 키별로 요소들의 갯수를 카운트 > Action

In [47]:
rdd9.countByKey()

defaultdict(int, {'짜장면': 2, '짬뽕': 2})

In [49]:
# key() 키만 가진 RDD > transformation
rdd9.keys()

PythonRDD[59] at RDD at PythonRDD.scala:53

In [50]:
rdd9.keys().collect()

['짜장면', '짬뽕', '짜장면', '짬뽕']

### join - outer join, inner join
inner-join : 두개의 집합에서 서로 함께 존재하는 요소(key)의 집합(default)<br>
outer-join : 한쪽에 있고, 다른쪽에 없을 때 - left, right

In [54]:
join_rdd1 = spark.parallelize(
    [
         ('a', 10),
         ('b', 20),
         ('c', 30)   
    ]
)

join_rdd2 = spark.parallelize(
    [
        ('a', 10),
        ('b', 20),
        ('b', 10)
    ]
)

In [55]:
join_rdd1.join(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 10))]

In [56]:
join_rdd1.leftOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('c', (30, None)), ('a', (10, 10))]

In [57]:
join_rdd1.rightOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 10))]